In [242]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import sqlite3

csv_file_path = 'heart.csv'

df = pd.read_csv(csv_file_path, delimiter=';')

In [244]:
database_name = 'heart.db'
conn = sqlite3.connect(database_name)

df.to_sql('patients', conn, if_exists='replace', index=False)

303

In [245]:
query = "SELECT * FROM patients"
db = pd.read_sql_query(query, conn)

db.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [246]:
db.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [247]:
db.shape

(303, 14)

In [249]:
print(df.dtypes)

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object


In [250]:
db.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [251]:
db.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [252]:
duplicates = db.duplicated()

db[duplicates].head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
164,38,1,2,138,175,0,1,173,0,0.0,2,4,2,1


In [253]:
no_duplicates = db.drop_duplicates()

no_duplicates.to_sql('patients', conn, if_exists='replace', index=False)

query = "SELECT * FROM patients"
db = pd.read_sql_query(query, conn)

duplicates = db.duplicated();
num_duplicates = duplicates.sum()
print(num_duplicates)

0


In [254]:
implausible_conditions = {
    'age': (db['age'] < 0) | (db['age'] > 120),
    'trestbps': (db['trestbps'] < 80) | (db['trestbps'] > 200),
    'chol': (db['chol'] < 100) | (db['chol'] > 600),
    'thalach': (db['thalach'] < 60) | (db['thalach'] > 220),
    'oldpeak': (db['oldpeak'] < 0)
}

for column, condition in implausible_conditions.items():
    invalid_rows = db[condition]
    if not invalid_rows.empty:
        print(f"Implausible values detected in {column}:")
        print(invalid_rows)
    else:
        print(f"No implausible values detected in {column}.")


No implausible values detected in age.
No implausible values detected in trestbps.
No implausible values detected in chol.
No implausible values detected in thalach.
No implausible values detected in oldpeak.


In [255]:
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

z_scores = np.abs(stats.zscore(db[numerical_cols]))
threshold = 3

outliers = (z_scores > threshold).any(axis=1)

outliers_df = db[outliers]
#outliers_df.head()

outliers_z_scores = pd.DataFrame(z_scores[outliers], columns=numerical_cols, index=outliers_df.index)
#print(outliers_z_scores)

details = outliers_df.copy()
for col in numerical_cols:
    details[col + '_zscore'] = outliers_z_scores[col]

details.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,age_zscore,trestbps_zscore,chol_zscore,thalach_zscore,oldpeak_zscore
28,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1,1.171205,0.478910,3.299932,0.324963,0.209608
85,67,0,2,115,564,0,0,160,0,1.6,1,0,3,1,1.392616,0.946867,6.145034,0.456164,0.480328
203,62,0,0,160,164,0,0,145,0,6.2,0,3,3,0,0.839089,1.619532,1.596741,0.199843,4.447460
219,63,0,0,150,407,0,0,154,0,4.0,1,3,3,0,0.949794,1.049221,3.106387,0.193761,2.550136
220,55,1,0,140,217,0,1,111,1,5.6,0,0,3,0,0.064151,0.478910,0.570956,1.686795,3.930008
